In [ ]:
# korekta błędu w Keras po zmianie w bibliotece numpy
import numpy as np
np_load_old = np.load
np.load = lambda *a, **k: np_load_old(*a, allow_pickle=True, **k)

# wyłączenie ostrzeżeń
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
import keras
keras.__version__

# Generowanie tekstu za pomocą sieci LSTM

[...]

## Implementacja algorytmu LSTM generującego tekst na poziomie liter


Czas skorzystać z pakietu Keras i zastosować teorię w praktyce. Na początek będziemy potrzebować dużo danych tekstowych do trenowania modelu języka. Możemy skorzystać z dowolnego wystarczająco rozbudowanego zestawu plików tekstowych — Wikipedii, Władcy pierścieni itd. W zaprezentowanym przykładzie posłużymy się wybranymi dziełami Friedricha Nietzschego — niemieckiego filozofa żyjącego w XIX w. — przetłumaczonymi na język angielski. W związku z tym wytrenujemy model odwzorowujący specyficzny styl pisania Nietzschego. Ponadto model ten będzie generował teksty tylko na wybrane tematy — nie będzie to ogólny model języka angielskiego.

## Przygotowanie danych

Zacznijmy od pobrania korpusu i zapisania go przy użyciu tylko małych liter:

In [ ]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))


Teraz dokonamy wyodrębnienia częściowo zachodzących na siebie sekwencji o długości maxlen, zakodujemy je techniką kodowania z gorącą jedynką, a następnie umieścimy je w trójwymiarowej tablicy Numpy o kształcie x (sequences, maxlen, unique_characters). Jednocześnie przygotujemy tablicę y zawierającą „wartości docelowe”, które w tym przypadku są po prostu literami umieszczanymi po każdej z wyodrębnionych sekwencji. Wartości te zostaną zapisane przy użyciu techniki kodowania z gorącą jedynką.

In [ ]:
# Wyodrębniamy sekwencje składajace się z 60 znaków.
maxlen = 60

# Nowa sekwencja jest próbkowana co 3 znaki.
step = 3

# Zmienna, w której zapisywane będą wyodrębnione sekwencje.
sentences = []

# Zmienna, w której zapisywane będą kolejne znaki (cele).
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Liczba sekwencji:', len(sentences))

# Lista unikatowych znaków wchodzących w skład korpusu.
chars = sorted(list(set(text)))
print('Liczba unikatowych znaków:', len(chars))
# Słownik przypisujące unikatowe znaki do ich indeksów.
char_indices = dict((char, chars.index(char)) for char in chars)

# Znaki są zapisywane w formie tablic binarnych przy użyciu kodowania z gorącą jedynką.
print('Tworzenie wektorów...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Budowanie sieci

Sieć składa się z pojedynczej warstwy LSTM, klasyfikatora Dense z funkcją aktywacji softmax. Pamiętajmy o tym, że generowanie danych sekwencyjnych nie musi być przeprowadzane przy użyciu rekurencyjnych sieci neuronowych. Ostatnio coraz częściej stosuje się w tym celu jednowymiarowe sieci konwolucyjne.

In [ ]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Wartości docelowe (znaki) są zakodowane przy użyciu techniki gorącej jedynki, a więc funkcją straty trenowanego modelu będzie categorical_crossentropy.

In [ ]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Trenowanie modelu języka i próbkowanie z niego


Dysponując wytrenowanym modelem i kawałkiem początkowego tekstu, możemy wygenerować nowy tekst. W tym celu należy powtarzać następujące operacje:

* 1) Użyj modelu w celu wygenerowania rozkładu prawdopodobieństwa następnego znaku kontynuującego obecny tekst.
* 2) Zmodyfikuj rozkład, korzystając z określonej wartości parametru temperature.
* 3) Przeprowadź operację losowego próbkowania następnego znaku na podstawie zmodyfikowanego rozkładu.
* 4) Dodaj nowy znak na końcu obecnego tekstu.

Oto kod używany do zmiany wag rozkładu prawdopodobieństwa wygenerowanego przez model. Kod ten tworzy funkcję próbkującą, która również określa indeks znaku:

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Na koniec poniższa pętla wykonuje operację trenowania modelu i generowania tekstu. Wygenerujemy teksty przy różnych wartościach parametru temperature (wartości te będą zmieniane przy rozpoczęciu kolejnych epok procesu trenowania). Pozwoli to nam zobaczyć, jak zmienia się tekst wraz z udoskonalaniem modelu, a także to, jak parametr temperature wpływa na strategię próbkowania.

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Jedna iteracja trenowania modelu na dostępnych danych treningowych.
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Losowanie tekstu początkowego.
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generowanie przy użyciu tekstu początkowego: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ Wartość parametru temperature:', temperature)
        sys.stdout.write(generated_text)

        # Generowanie 400 znaków (proces rozpoczyna się od wylosowanego tekstu początkowego).
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


Jak widać, niska wartość parametru temperature prowadzi do uzyskania tekstu, który charakteryzuje się dużą przewidywalnością i powtarzalnością, ale jego lokalna struktura jest bardzo realistyczna — wszystkie wygenerowane słowa (słowo jest lokalnym wzorcem składającym się ze znaków) występują w języku angielskim. Przy wyższych wartościach parametru temperature wygenerowany tekst staje się bardziej interesujący, zaskakujący, a nawet kreatywny — algorytm czasami wymyśla nawet nowe słowa, które brzmią tak, jakby były naprawdę istniejącymi słowami (są to np. eterned i troveration), ale lokalna struktura tekstu zaczyna się załamywać i większość słów wygląda tak, jakby była prawie losowym zbiorem znaków. Bez wątpienia najciekawsze efekty w przypadku tego generowania tekstu uzyskuje się przy parametrze temperature równym 0,5. Zawsze warto eksperymentować z różnymi strategiami próbkowania! Dobra równowaga między wytrenowaną strukturą a losowością sprawi, że wygenerowany tekst będzie interesujący.

Trenując model dłużej, tworząc większy model i stosując większy zbiór danych, można generować próbki, które wyglądają o wiele składniej i bardziej realistycznie. Oczywiście nie należy oczekiwać od modelu wygenerowania tekstu, który będzie miał jakiś większy sens — mechanizm generujący tekst tylko próbkuje litery z modelu statystycznego określającego ich kolejność. Język jest kanałem komunikacji, a rozmowy dotyczące różnych tematów charakteryzują się inną strukturą statystyczną. Tezę tę można udowodnić, odpowiadając sobie na pytanie: co, jeżeli język ludzki zostałby skompresowany tak, jak kompresowana jest większość cyfrowej komunikacji między komputerami? Wówczas język przenosiłby tyle samo informacji, ale nie charakteryzowałby się żadną ukrytą strukturą statystyczną, co uniemożliwiłoby wytrenowanie modelu języka w sposób, w jaki zrobiliśmy to przed chwilą.


## Wnioski

* Dyskretna sekwencja danych może zostać wygenerowana poprzez trenowanie modelu pod kątem przewidywania kolejnych elementów tekstu na podstawie wcześniejszego ciągu znaków.
* Model trenowany na zbiorze danych tekstowych określany jest mianem modelu języka. Może on być oparty na słowach lub literach.
* Próbkowanie zbioru elementów tekstu wymaga kompromisu między bezkrytycznym przyjmowaniem przewidywań modelu a losowością.
* Można to zrobić przy użyciu parametru temperature funkcji softmax. Wybór właściwej wartości tego parametru powinien zostać dokonany na drodze eksperymentów.